In [14]:
# Importing needed libraries
import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval 
#module that converts a string of lists to a normal list

In [15]:
df = pd.read_csv('Hotel_reviews.csv')
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [16]:
# Replacing 'united kingdom' with 'UK' for easy use
df.Hotel_Address = df.Hotel_Address.str.replace('United Kingdom','UK')

In [17]:
# Splitting the hotel address and picking out the last string which would be the countries
df['countries'] = df.Hotel_Address.apply(lambda x: x.split(' ')[-1])

In [18]:
# All the hotels are located in six(6) countries 
df.countries.unique() 

array(['Netherlands', 'UK', 'France', 'Spain', 'Italy', 'Austria'],
      dtype=object)

In [19]:
# Dropping unneeded columns
df.drop(['Additional_Number_of_Scoring',
       'Review_Date','Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng'],1,inplace=True)

C:\Users\albrt\AppData\Local\Temp\ipykernel_10656\156294051.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Additional_Number_of_Scoring',


In [20]:
#module that converts a string of lists to a normal list
from ast import literal_eval
#function to convert array of tags to string
def impute(col):
  col = col[0]
  if (type(col) != list):
    return "".join(literal_eval(col))
  else:
    return col

In [21]:
#using the function
df['Tags']  = df[['Tags']].apply(impute,axis=1)
df.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Family with young children Dup...,Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Solo traveler Duplex Double Ro...,Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Suite Stayed 2 nights ...,Netherlands


In [22]:
def Input_your_destination_and_description(location,description):
    # Making these columns lowercase
    df['countries']=df['countries'].str.lower()
    df['Tags']=df['Tags'].str.lower()
    
    # Dividing the texts into small tokens (sentences into words)
    description = description.lower()
    description_tokens=word_tokenize(description)  
    
    sw = stopwords.words('english') # List of predefined english  stopwords to be used for computing
    lemm = WordNetLemmatizer() # We now define the functions below connecting these imported packages
    filtered_sen = {w for w in description_tokens if not w in sw}
    f_set=set()
    for fs in filtered_sen:
        f_set.add(lemm.lemmatize(fs))
    
    
    # Defining a new variable that takes in the location inputted and bring out the features defined below
    country_feat = df[df['countries']==location.lower()]
    country_feat = country_feat.set_index(np.arange(country_feat.shape[0]))
    l1 =[];l2 =[];cos=[];
    for i in range(country_feat.shape[0]):
        temp_tokens=word_tokenize(country_feat['Tags'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
       

        cos.append(len(rvector))
    country_feat['similarity']=cos
    country_feat=country_feat.sort_values(by='similarity',ascending=False)
    country_feat.drop_duplicates(subset='Hotel_Name',keep='first',inplace=True)
    country_feat.sort_values('Average_Score',ascending=False,inplace=True)
    country_feat.reset_index(inplace=True)
    return country_feat[['Hotel_Name','Average_Score','Hotel_Address']].head(10)

In [28]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\albrt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\albrt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\albrt\AppData\Roaming\nltk_data...


True

In [29]:
Input_your_destination_and_description('Netherlands','I am going on a business trip, I need a standard room and i am staying for two nights ')

,Hotel_Name,Average_Score,Hotel_Address
0,Waldorf Astoria Amsterdam,9.5,Herengracht 542 556 Amsterdam City Center 1017...
1,The Toren,9.4,Keizersgracht 164 Amsterdam City Center 1015 C...
2,Pillows Anna van den Vondel Amsterdam,9.4,Anna van den Vondelstraat 6 Oud West 1054 GZ A...
3,Luxury Suites Amsterdam,9.3,Oudeschans 75 Amsterdam City Center 1011 KW Am...
4,The Hoxton Amsterdam,9.3,Herengracht 255 Amsterdam City Center 1016 BJ ...
5,Ambassade Hotel,9.3,Herengracht 341 Amsterdam City Center 1016 AZ ...
6,Canal House,9.3,Keizersgracht 148 Amsterdam City Center 1015 C...
7,Andaz Amsterdam Prinsengracht A Hyatt Hotel,9.3,Prinsengracht 587 Amsterdam City Center 1067 H...
8,Banks Mansion All Inclusive Hotel,9.2,Herengracht 519 525 Amsterdam City Center 1017...
9,Conservatorium Hotel,9.2,Van Baerlestraat 27 Oud Zuid 1071 AN Amsterdam...


In [30]:
Input_your_destination_and_description('UK','I want to go on a business trip with my employees')

,Hotel_Name,Average_Score,Hotel_Address
0,41,9.6,41 Buckingham Palace Road Westminster Borough ...
1,Haymarket Hotel,9.6,1 Suffolk Place Westminster Borough London SW1...
2,Charlotte Street Hotel,9.5,15 17 Charlotte Street Hotel Westminster Borou...
3,Taj 51 Buckingham Gate Suites and Residences,9.5,Buckingham Gate Westminster Borough London SW1...
4,The Soho Hotel,9.5,4 Richmond Mews Westminster Borough London W1D...
5,Milestone Hotel Kensington,9.5,1 Kensington Court Kensington and Chelsea Lond...
6,Ham Yard Hotel,9.5,One Ham Yard Westminster Borough London W1D 7D...
7,The Lanesborough,9.4,Hyde Park Corner Westminster Borough London SW...
8,Lansbury Heritage Hotel,9.4,117 Poplar High Street Tower Hamlets London E1...
9,The Goring,9.4,15 Beeston Place Westminster Borough London SW...
